In [17]:
from pathlib import Path

import numpy as np
from jax import (grad, Array)
import jax.numpy as jnp
from scipy.interpolate import (make_lsq_spline, BSpline, Akima1DInterpolator, FloaterHormannInterpolator)
from statsmodels.nonparametric.kernel_regression import KernelReg
import statsmodels.api as sm
import polars as pl
import plotly.express as px

# Exploratory Data Analysis and Feature Engineering

We have data on model_year, calendar_time, and advertised_inventory.

In [2]:
# Import data
mixed_years_path = Path("..", "raw_data", "model_year_vs_calendar_time", "model_year_vs_calendar_time.csv")
mixed_years = pl.scan_csv(
    source=mixed_years_path,
)
# View head of data
mixed_years.collect().head()

model_year,calendar_time,advertised_inventory,model_year_is_previous_calendar_year,model_year_is_calendar_year,model_year_is_next_calendar_year
i64,f64,f64,i64,i64,i64
2022,2020.5,0.0,0,0,0
2022,2020.9,0.0,0,0,0
2022,2022.420572,935.612174,0,1,0
2022,2022.441428,928.768412,0,1,0
2022,2022.462284,920.898087,0,1,0


Create a new variable called yr_diff to represent how far ahead the calendar_time is from the model_year.

In [3]:
mixed_years = (
    mixed_years
        .select(
            "model_year",
            # Use the midpoint rule.
            (pl.col("calendar_time") - (pl.col("model_year") + 6/12)).alias("yr_diff"),
            "advertised_inventory"
        )
)

# Also, pad in some more 0s for easier modeling later.
mixed_years = (mixed_years
    .collect()
    .vstack(
        pl.DataFrame(
            {
                "model_year": [2022]*10, 
                "yr_diff": np.linspace(2.05, 2.19, 10),
                "advertised_inventory": [0.0]*10,
            }
        )
    )
    .vstack(
        pl.DataFrame(
            {
                "model_year": [2023]*10, 
                "yr_diff": [1.95, 2.0, 2.05, -1.65, -1.70, -1.75, -1.8, -1.85, -1.9, -1.95],
                "advertised_inventory": [0.0]*10,
            }
        )
    )
    .lazy()
)

Except for some fluctations in year-to-year amplitudes, the pattern of inventory rise and fall for different model years appears remarkably consistent.  Inventories for each model year peak around July of the same calendar year.

In [4]:
px.scatter(
    x=mixed_years.select("yr_diff").collect().to_series(),
    y=mixed_years.select("advertised_inventory").collect().to_series(),
    color=mixed_years.select(pl.col("model_year").cast(pl.Utf8)).collect().to_series(),
    title="The Rise and Fall of Vehicle Inventories",
    labels={
        "x": "calendar_time - (model_year + 6/12) (years)",
        "y": "Advertised Vehicle Inventory (thousands)",
        "color": "Model Year"
    }
)

/home/justin/bin/mambaforge/envs/justins_room/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


# Predict the Average Inventory Derivative

We will combine our data from the different model years to predict an average inventory derivative that can be used for prior model years. 

In [5]:
# For each model year that we have data (2022 through 2025),
# fit a separate spline regression.  Do this so that monthly
# inventory counts can be predicted despite not having
# direct access to this information.  (All of the information
# that we have to go on is the graph from www.spglobal.com/mobility)
# Use the splines to interpolate values.  

model_yr_range = range(2022, 2026, 1)
inventory_model = dict.fromkeys(model_yr_range)

# Create some evenly space yr_diffs for evaluating each model.
# These same yr_diffs can be used for other model years
# to compare apples to apples.
step = 1.0/1200.0
a = -3.0
b = 3.0

yr_diffs_for_prediction = np.linspace(a, b, num=int((b - a)/step + 1))

# For each year, train and predict on only some of the yr_diffs.
yr_diff_filter = {
    2022: (-0.079428, 2.19),
    2023: (-2.0, 2.1),
    2024: (-2.0, 0.283906),
    2025: (-2.0, -0.716094)
}

# Create a data-frame to store the predictions from each model.
mixed_years_2 = pl.DataFrame()

# Get plotly figures set up before the loop.
fig = px.scatter()
fig_2 = px.scatter()

# Make a model for each model_year and fit the model.
for yr in model_yr_range:
    # For the current yr, get the yr_diffs to train and predict on.
    a, b = yr_diff_filter[yr]

    # Get the yr_diffs for predicting.
    yr_diffs_for_prediction_current = yr_diffs_for_prediction[(yr_diffs_for_prediction >= a) & (yr_diffs_for_prediction <= b)]
    
    # Get the yr_diffs for training.
    yr_diffs_for_training = (mixed_years
        .filter((pl.col("yr_diff").is_between(a, b)) & (pl.col("model_year") == yr))
        .sort(by=pl.col("yr_diff"))
        .select("yr_diff")
        .collect()
        .to_series()
        .to_numpy()
    )

    # Get the response values for training.
    y = (mixed_years
        .filter((pl.col("yr_diff").is_between(a, b)) & (pl.col("model_year") == yr))
        .sort(by=pl.col("yr_diff"))
        .select("advertised_inventory")
        .collect()
        .to_series()
        .to_numpy()
    )
    
    # Prepare to fit model.
    # Fit model.
    current_yr_inventory_model = Akima1DInterpolator(
        x=yr_diffs_for_training,
        y=y,
        method="makima",
        extrapolate=True
    )

    # Get the predictions.
    preds = current_yr_inventory_model(yr_diffs_for_prediction_current)

    # Replace negative values generated by the spline regression.
    # (I cannot figure out how to put constraints on the spline regression
    # other than to just fix the predictions like I do here.)
    preds = np.where(preds < 0, 0, preds)
    
    derivatives = current_yr_inventory_model(
        x=yr_diffs_for_prediction_current, 
        nu=1
    )

    # Save the fitted model.
    inventory_model[yr] = current_yr_inventory_model

    # Put the predictions from current_yr_inventory_model into a new data-frame.
    mixed_years_current_preds = pl.LazyFrame(
        data={
            "yr_diff": yr_diffs_for_prediction_current,
            "model_year": [yr]*len(yr_diffs_for_prediction_current),
            "predicted_inventory": preds,
            "predicted_inventory_derivative": derivatives
        },
        schema={
            "yr_diff": pl.Float64,
            "model_year": pl.Int32,
            "predicted_inventory": pl.Float64,
            "predicted_inventory_derivative": pl.Float64
        }
    )
    
    # Save mixed_years_current_preds for the current model_year.
    mixed_years_2 = (mixed_years_2
        .vstack(mixed_years_current_preds.collect())     
    )

    ########################################################
    # Figures
    ########################################################
    # Make a plot of the original training data alongside the predictions.
    fig.add_scatter(
        x=(mixed_years_2
           .filter(pl.col("model_year") == yr)
           .select("yr_diff")
           .to_series()
        ),
        y=(mixed_years_2
           .filter(pl.col("model_year") == yr)
           .select("predicted_inventory")
           .to_series()
        ),
        mode="markers",
        name=f"{yr} Predicted"
    )

    fig.add_scatter(
        x=(mixed_years
           .filter(pl.col("model_year") == yr)
           .select("yr_diff")
           .collect()
           .to_series()
        ),
        y=(mixed_years
           .filter(pl.col("model_year") == yr)
           .select("advertised_inventory")
           .collect()
           .to_series()
        ),
        mode="markers",
        name=f"{yr} Actual"
    )

    # https://plotly.com/python/figure-labels/
    fig.update_layout(
        title=dict(text="Curve Fitting for Different Model Years"),
        xaxis=dict(
            title=dict(
                text="calendar_time - (model_year + 6/12) (years)"
            )
        ),
        yaxis=dict(
            title=dict(
                text="Advertised Vehicle Inventory (thousands)"
            )
        )
    )

    fig_2.add_scatter(
        x=(mixed_years_2
           .filter(pl.col("model_year") == yr)
           .select("yr_diff")
           .to_series()
        ),
        y=(mixed_years_2
           .filter(pl.col("model_year") == yr)
           .select("predicted_inventory_derivative")
           .to_series()
        ),
        mode="markers",
        name=f"{yr}"
    )

    fig_2.update_layout(
        title=dict(text="Predicted Inventory Derivatives for Different Model Years"),
        xaxis=dict(
            title=dict(
                text="calendar_time - (model_year + 6/12) (years)"
            )
        ),
        yaxis=dict(
            title=dict(
                text="Predicted d[Advertised Vehicle Inventory]/dt (thousands/yr)"
            )
        ),
        legend_title_text="Model Year"
    )

# Show the figures.
fig.show()
fig_2.show()

In [97]:
# Determine how to scale the inventory curve for model
# year 2023 to best match the inventory curve
# of model years 2022 and 2024.  
# Use the results to impute for model years 2022 and 2024.
# Determine the best scaling of the imputed curve for model year 2024 to
# match the curve for model year 2025.
# Use the reults to impute for model year 2025.
def raised_cosine_pdf(x: Array|float, mu: float=0.0, s: float=1.0):
    """https://en.m.wikipedia.org/wiki/Raised_cosine_distribution"""
    non_zero_pdf_val = 1.0/(2.0*s)*(1 + jnp.cos(np.pi * (x - mu)/s))
    pdf_vals = jnp.where(jnp.absolute(x - mu) > s, 0, non_zero_pdf_val)
    return pdf_vals

def inventory_scaling_func(yr_diffs, a: float=1.0, mu: float=0.0, s: float=1.0):
    return 1.0 + a*raised_cosine_pdf(x=yr_diffs, mu=mu, s=s)

def predict(yr_diffs, inventory, a: float=1.0, mu: float=0.0, s: float=1.0):
    return inventory * inventory_scaling_func(yr_diffs=yr_diffs, a=a, mu=mu, s=s)

def mse(predicted: Array, actual: Array):
    n = len(actual)
    error = actual - predicted
    return (1.0/n * jnp.dot(error, error))

def loss(params: dict, yr_diffs, inventory, actual, constrained_index):
    """
    Args:
        params: Dictionary with keys of `a`, `mu`, and `s`.
        constrained_index: either a 0 or a -1.  Used to indicate the
            entries in yr_diffs and inventory which should be predicted
            precisely.
    """
    predicted = predict(
        yr_diffs=yr_diffs,
        inventory=inventory,
        **params
    )

    constraint_penalty = (actual[constrained_index] - predicted[constrained_index])**2
    other_loss = mse(predicted=predicted, actual=actual)

    return constraint_penalty + other_loss


grad_mse = grad(mse)

def update():
    ...

In [77]:
params = {
    "predicted": np.array([3]),
    "actual": np.array([2.6])
}

mse(**params)

Array(0.16000001, dtype=float32)

In [99]:
yr_diffs=(mixed_years_2
    .filter((pl.col("yr_diff").is_between(*yr_diff_filter[2022])) & (pl.col("yr_diff").is_between(*yr_diff_filter[2023])) & (pl.col("model_year") == 2022))
    .select("yr_diff")
    .to_series()
    .to_numpy()
)

inventory = (mixed_years_2
    .filter((pl.col("yr_diff").is_between(*yr_diff_filter[2022])) & (pl.col("model_year") == 2023))
    .select("predicted_inventory")
    .to_series()
    .to_numpy()
)

actual = (mixed_years_2
    .filter((pl.col("yr_diff").is_between(*yr_diff_filter[2022])) & (pl.col("yr_diff").is_between(*yr_diff_filter[2023])) & (pl.col("model_year") == 2022))
    .select("predicted_inventory")
    .to_series()
    .to_numpy()
)

params = {
    "a": -1.8,
    "mu": -0.15,
    "s": 4.5
}

preds = predict(
    yr_diffs=yr_diffs,
    inventory=inventory,
    **params
)

print(loss(
    params=params,
    yr_diffs=yr_diffs,
    inventory=inventory,
    actual=actual,
    constrained_index=0
))

fig = px.scatter()

fig.add_scatter(
    x=(mixed_years_2
        .filter(pl.col("model_year") == 2023)
        .select("yr_diff")
        .to_series()
    ),
    y=(mixed_years_2
        .filter(pl.col("model_year") == 2023)
        .select("predicted_inventory")
        .to_series()
    ),
    mode="markers",
    name=f"2023 Predicted"
)

fig.add_scatter(
    x=(mixed_years_2
        .filter(pl.col("model_year") == 2022)
        .select("yr_diff")
        .to_series()
    ),
    y=actual,
    mode="markers",
    name=f"2022 Predicted"
)

fig.add_scatter(
    x=yr_diffs,
    y=preds
)

fig.show()

829.5037


In [15]:
fig = px.scatter()
yr_diffs_to_use = dict()
rel_dist_from_2023_pred_inventory = dict()

for yr in range(2022, 2026, 1):
    # We can only compare to 2023 at common times.
    yr_diffs_to_use[yr] = {
        "for_comparison": (mixed_years_2
            .filter((pl.col("model_year") == 2023) & pl.col("yr_diff").is_between(
                max(yr_diff_filter[yr][0], yr_diff_filter[2023][0]), 
                min(yr_diff_filter[yr][1], yr_diff_filter[2023][1])))
            .select("yr_diff")
            .to_series()
            .to_numpy()
        ),
        # 2022 uses 2024
        # 
        "for_prediction": (mixed_years_2
            .filter((pl.col("model_year") == 2023) & pl.col("yr_diff").is_between(
                max(yr_diff_filter[yr][0], yr_diff_filter[2023][0]), 
                min(yr_diff_filter[yr][1], yr_diff_filter[2023][1])))
            .select("yr_diff")
            .to_series()
            .to_numpy()
        )
    }
    
    
    
    
    # Calculate the relative distance from model year 2023's predicted inventory
    # at each time point.
    preds_for_2023_to_use = (mixed_years_2
        .filter((pl.col("model_year") == 2023) & pl.col("yr_diff").is_in(yr_diffs_to_use[yr]))
        .select("predicted_inventory")
        .to_series()
    )

    rel_dist_from_2023_pred_inventory[yr] = np.absolute(
        np.subtract(
            preds_for_2023_to_use, 
            mixed_years_2
                .filter((pl.col("model_year") == yr) & pl.col("yr_diff").is_in(yr_diffs_to_use[yr]))
                .select("predicted_inventory")
                .to_series()
        )
    ) / (preds_for_2023_to_use + 200)
    
    fig.add_scatter(
        x=yr_diffs_to_use[yr],
        y=rel_dist_from_2023_pred_inventory[yr],
        mode="markers",
        name=f"{yr} Predicted"
    )
fig.show()

InvalidOperationError: 'is_in' cannot check for String values in Float64 data

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'filter' failed <---
FILTER [([(col("model_year")) == (2023)]) & (col("yr_diff").is_in([Series]))] FROM
  DF ["yr_diff", "model_year", "predicted_inventory", "predicted_inventory_derivative"]; PROJECT */4 COLUMNS

In [ ]:
rel_dist_from_2023_pred_inventory_2024_sub = rel_dist_from_2023_pred_inventory[2024].filter(yr_diffs_to_use[2024] >= yr_diffs_to_use[2022][0]).to_numpy()
rel_dist_from_2023_pred_inventory_2022_sub = rel_dist_from_2023_pred_inventory[2022].filter(yr_diffs_to_use[2022] <= yr_diffs_to_use[2024][-1]).to_numpy()
yr_diffs_to_use_2024_sub = yr_diffs_to_use[2024].filter(yr_diffs_to_use[2024] >= yr_diffs_to_use[2022][0]).to_numpy()

rel_dist_from_2023_pred_inventory_2024_2022_avg = np.mean(
    rel_dist_from_2023_pred_inventory_2024_sub - rel_dist_from_2023_pred_inventory_2022_sub
)

np.float64(0.1838212070033685)

In [79]:
def normal_kernel(x: np.ndarray | float, mu: float=0.0, sigma: float=1.0) -> np.ndarray | float:
    """
    Parameters:
        
    """
    return np.exp(-((x - mu)**2)/(2 * sigma**2))

def weighted_avg(a:float, b:float, distances_from_a: np.ndarray | float, mu=0.0, sigma=1.0):
    a_weights = normal_kernel(x=distances_from_a, mu=mu, sigma=sigma)
    b_weights = 1 - a_weights

    return a * a_weights + b * b_weights

In [89]:
rel_dist_from_2023_pred_inventory_2024_sub_last = rel_dist_from_2023_pred_inventory_2024_sub[-1]
yr_diffs = yr_diffs_to_use[2022].filter(yr_diffs_to_use[2022] > yr_diffs_to_use[2024][-1]).to_numpy()
distances_from_a = yr_diffs - min(yr_diffs)
avgs = weighted_avg(
    a=rel_dist_from_2023_pred_inventory_2024_sub_last,
    b=rel_dist_from_2023_pred_inventory_2024_2022_avg,
    distances_from_a=distances_from_a
)
    
a = preds_for_2023_to_use.filter(yr_diffs_to_use[2023] > yr_diffs_to_use[2024][-1]).to_numpy()
b = a - avgs*(a + 200)
b

ShapeError: filter's length: 4920 differs from that of the series: 1541

In [58]:
rel_dist_from_2023_pred_inventory_2024_sub = rel_dist_from_2023_pred_inventory[2024].filter(yr_diffs_to_use[2024] >= yr_diffs_to_use[2022][0]).to_numpy()
rel_dist_from_2023_pred_inventory_2022_sub = rel_dist_from_2023_pred_inventory[2022].filter(yr_diffs_to_use[2022] <= yr_diffs_to_use[2024][-1]).to_numpy()
yr_diffs_to_use_2024_sub = yr_diffs_to_use[2024].filter(yr_diffs_to_use[2024] >= yr_diffs_to_use[2022][0]).to_numpy()

X = np.column_stack((
    np.ones(shape=len(yr_diffs_to_use_2024_sub)),
    yr_diffs_to_use_2024_sub,
    rel_dist_from_2023_pred_inventory_2024_sub,
    # yr_diffs_to_use_2024_sub * rel_dist_from_2023_pred_inventory_2024_sub
))

res = sm.OLS(
    endog=rel_dist_from_2023_pred_inventory_2022_sub,
    exog=X
).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.842
Model:                            OLS   Adj. R-squared:                  0.841
Method:                 Least Squares   F-statistic:                     1155.
Date:                Wed, 02 Apr 2025   Prob (F-statistic):          2.86e-174
Time:                        10:09:49   Log-Likelihood:                 1475.2
No. Observations:                 436   AIC:                            -2944.
Df Residuals:                     433   BIC:                            -2932.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.4539      0.051      8.878      0.000       0.353       0.554
x1            -0.1918      0.008    -25.309      0.000      -0.207      -0.177
x2            -0.1612      0.095     -1.701      0.090      -0.347       0.025
==============================================================================
Omnibus:                       42.662   Durbin-Watson:                   0.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               53.202
Skew:                          -0.848   Prob(JB):                     2.80e-12
Kurtosis:                       2.767   Cond. No.                         311.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
yr_diffs_to_use_2024_sub_2 = yr_diffs_to_use[2024].filter(yr_diffs_to_use[2024] < yr_diffs_to_use[2022][0]).to_numpy()
rel_dist_from_2023_pred_inventory_2024_sub_2 = rel_dist_from_2023_pred_inventory[2024].filter(yr_diffs_to_use[2024] < yr_diffs_to_use[2022][0]).to_numpy()

X = np.column_stack((
    np.ones(shape=len(yr_diffs_to_use_2024_sub_2)),
    yr_diffs_to_use_2024_sub_2,
    rel_dist_from_2023_pred_inventory_2024_sub_2,
    # yr_diffs_to_use_2024_sub_2 * rel_dist_from_2023_pred_inventory_2024_sub_2
))

rel_dist_from_2023_pred_inventory_2022_pred = res.predict(exog=X)

In [60]:
px.scatter(
    x=np.concat((yr_diffs_to_use_2024_sub_2, yr_diffs_to_use_2024_sub)),
    y=np.concat((rel_dist_from_2023_pred_inventory_2022_pred, rel_dist_from_2023_pred_inventory_2022_sub))
)

In [ ]:
# Do kernel regression to get an average across all model years.
derivative_model_2 = KernelReg(
    endog=mixed_years_2.select("predicted_inventory_derivative").to_series().to_numpy(),
    exog=mixed_years_2.select("yr_diff").to_series().to_numpy(),
    var_type="c",
    reg_type="ll",
    bw=[0.04]
)
predicted_inventory_derivative, _ = derivative_model_2.fit()

In [20]:
fig = px.scatter()
fig.add_scatter(
    x=(mixed_years_2
        .select("yr_diff")
        .to_series()
    ),
    y=predicted_inventory_derivative,
    mode="markers",
    name=f"{yr}"
)

fig.update_layout(
    title=dict(text="Predicted Average Inventory Derivative"),
    xaxis=dict(
        title=dict(
            text="calendar_time - (model_year + 6/12) (years)"
        )
    ),
    yaxis=dict(
        title=dict(
            text="Predicted d[Advertised Vehicle Inventory]/dt (thousands/yr)"
        )
    )
)

# Show the figure
fig.show()

In [ ]:
np.trapezoid(y=mixed_years_2
           .filter(pl.col("model_year") == 2023)
           .select("predicted_inventory_derivative")
           .to_series().to_numpy(), dx=step)

np.float64(0.0018605435097924783)

In [ ]:
# Make sure that the predicted inventory derivative integrates to 0.


Now, adjust for the fact that in recent years, model years have been released earlier and earlier.  See [link](https://en.wikipedia.org/wiki/Model_year#:~:text=Model%20year%20followed%20with%20calendar,that%20the%20vehicle%20was%20manufactured.).

In 1935, the derivative of inventory can become positive at `calendar_time - (model_year + 6/12) = -0.417`.  In 1965, the derivative of inventory can become positive at `calendar_time - (model_year + 6/12) = -1.167`.

In [28]:
fig = px.scatter()
fig.add_scatter(
    x=(mixed_years_2
        .select("yr_diff")
        .to_series()
        + 0.5
    ),
    y=predicted_inventory_derivative,
    mode="markers",
    name=f"{yr}"
)

fig.update_layout(
    title=dict(text="Predicted Average Inventory Derivative (Adjusted)"),
    xaxis=dict(
        title=dict(
            text="calendar_time - (model_year + 6/12) (years)"
        )
    ),
    yaxis=dict(
        title=dict(
            text="Predicted d[Advertised Vehicle Inventory]/dt (thousands/yr)"
        )
    )
)

# Show the figure
fig.show()